# Notebook: Aplicar funciones a todo un DataFrame (pandas)
Este cuaderno demuestra técnicas para aplicar funciones sobre un DataFrame completo usando `applymap`, `apply`, operaciones vectorizadas y `np.where`. Incluye manejo de valores faltantes, medición de rendimiento y encadenamiento con `pipe`.



# 1) Importar librerías requeridas
Importar `pandas` y `numpy`; configurar opciones de visualización para ver más filas/columnas.


In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

# 2) Crear DataFrame de ejemplo
Construir un DataFrame con números, textos y valores NaN para casos de prueba.

In [2]:
df = pd.DataFrame({
    "numeros": [1, 2, 3, np.nan, 5, 10],
    "textos": ["a", "b", np.nan, "c", "d", "e"],
    "mixtos": ["10", 20, np.nan, "-5", "N/A", 0],
})
df

,numeros,textos,mixtos
0,1.0,a,10
1,2.0,b,20
2,3.0,NaN,NaN
3,NaN,c,-5
4,5.0,d,N/A
5,10.0,e,0


# 3) Aplicar función a todas las celdas con applymap
Definir una función segura y aplicar con `df.applymap` para transformar cada celda.

In [1]:
def safe_transform(x):
    if pd.isna(x):
        return x
    try:
        # Si es texto con dígitos, convertir a int; si es número, multiplicar por 2
        if isinstance(x, str):
            x_clean = x.replace("N/A", "")
            return int(x_clean) if x_clean.strip() != "" and x_clean.strip().lstrip("-+").isdigit() else x.upper()
        if isinstance(x, (int, float, np.integer, np.floating)):
            return x * 2
    except Exception:
        return x
    return x

df_mapped = df.map(safe_transform)
df_mapped

NameError: name 'df' is not defined

# 4) Aplicar función a todas las columnas con apply
Usar `df.apply(func, axis=0)` para operar columna por columna (e.g., normalización numérica).

In [4]:
def normalize_col(col):
    # Solo normaliza columnas numéricas; preserva otras
    if pd.api.types.is_numeric_dtype(col):
        return (col - col.min()) / (col.max() - col.min())
    return col

df_col_apply = df.apply(normalize_col, axis=0)
df_col_apply

,numeros,textos,mixtos
0,0.000000,a,10
1,0.111111,b,20
2,0.222222,NaN,NaN
3,NaN,c,-5
4,0.444444,d,N/A
5,1.000000,e,0


# 5) Aplicar función a todas las filas con apply (axis=1)
Usar `df.apply(func, axis=1)` para crear nuevas columnas basadas en lógica fila a fila.

In [5]:
def row_logic(row):
    # Crea una bandera si 'numeros' > 5 y 'textos' no es NaN
    cond = (pd.notna(row.get("numeros")) and row.get("numeros", 0) > 5) and pd.notna(row.get("textos"))
    return cond

df_rows = df.copy()
df_rows["flag"] = df.apply(row_logic, axis=1)
df_rows

,numeros,textos,mixtos,flag
0,1.0,a,10,False
1,2.0,b,20,False
2,3.0,NaN,NaN,False
3,NaN,c,-5,False
4,5.0,d,N/A,False
5,10.0,e,0,True


# 6) Vectorizar operaciones para todo el DataFrame
Reemplazar `apply` por operaciones vectorizadas de pandas/numpy para mayor rendimiento.

In [6]:
df_vec = df.copy()
# Vectorizar: multiplicar números por 2 sin apply
df_vec.loc[:, "numeros"] = df_vec["numeros"] * 2

# Vectorizar: upper() sobre textos (manejo de NaN con fillna y astype(str))
df_vec.loc[:, "textos"] = df_vec["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})

# Vectorizar: convertir strings numéricos en 'mixtos' a números, dejando lo demás igual
mixtos_num = pd.to_numeric(df_vec["mixtos"], errors="coerce")
df_vec.loc[:, "mixtos"] = np.where(pd.notna(mixtos_num), mixtos_num, df_vec["mixtos"])
df_vec

,numeros,textos,mixtos
0,2.0,A,10.0
1,4.0,B,20.0
2,6.0,NaN,NaN
3,NaN,C,-5.0
4,10.0,D,N/A
5,20.0,E,0.0


# 7) Manejo de valores faltantes antes de aplicar
Detectar y tratar NaN con `fillna`, `dropna` o máscaras; evitar errores en funciones.

In [7]:
df_na = df.copy()
# Reemplazar NaN en 'numeros' por 0 antes de cálculos
df_na["numeros_filled"] = df_na["numeros"].fillna(0)

# Eliminar filas completamente vacías (si aplica)
df_na_drop = df_na.dropna(how="all")
df_na, df_na_drop.shape

(   numeros textos mixtos  numeros_filled
 0      1.0      a     10             1.0
 1      2.0      b     20             2.0
 2      3.0    NaN    NaN             3.0
 3      NaN      c     -5             0.0
 4      5.0      d    N/A             5.0
 5     10.0      e      0            10.0,
 (6, 4))

# 8) Medir rendimiento con timeit en VS Code
Comparar tiempos entre `applymap`/`apply` y vectorización usando la celda `%%timeit` y revisar el panel de salida.

In [9]:
%%timeit
df.map(safe_transform)

339 μs ± 4.21 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit
# Vectorizado comparable a safe_transform (parcial)
_df = df.copy()
_df.loc[:, "numeros"] = _df["numeros"] * 2
_df.loc[:, "textos"] = _df["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})
mixtos_num = pd.to_numeric(_df["mixtos"], errors="coerce")
_df.loc[:, "mixtos"] = np.where(pd.notna(mixtos_num), mixtos_num, _df["mixtos"])

1.23 ms ± 6.13 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# 9) Usar np.where en todo el DataFrame
Aplicar condiciones globales con `np.where` para asignaciones rápidas en múltiples columnas.

In [11]:
df_where = df.copy()
cond_numeros_altos = df_where["numeros"].fillna(-np.inf) > 5
df_where["categoria"] = np.where(cond_numeros_altos, "alto", "bajo")
df_where

,numeros,textos,mixtos,categoria
0,1.0,a,10,bajo
1,2.0,b,20,bajo
2,3.0,NaN,NaN,bajo
3,NaN,c,-5,bajo
4,5.0,d,N/A,bajo
5,10.0,e,0,alto


# 10) Encadenar transformaciones con pipe
Definir funciones puras y encadenarlas con `df.pipe` para flujos reproducibles.

In [12]:
def transform_numeros(df):
    df = df.copy()
    df.loc[:, "numeros"] = df["numeros"].fillna(0) * 2
    return df

def upper_textos(df):
    df = df.copy()
    df.loc[:, "textos"] = df["textos"].fillna("").astype(str).str.upper().replace({"": np.nan})
    return df

def categorize(df):
    df = df.copy()
    df.loc[:, "categoria"] = np.where(df["numeros"] > 5, "alto", "bajo")
    return df

df_piped = (
    df.pipe(transform_numeros)
      .pipe(upper_textos)
      .pipe(categorize)
)
df_piped

,numeros,textos,mixtos,categoria
0,2.0,A,10,bajo
1,4.0,B,20,bajo
2,6.0,NaN,NaN,alto
3,0.0,C,-5,bajo
4,10.0,D,N/A,alto
5,20.0,E,0,alto
